# Day 1: Nsight Compute - The Kernel Microscope 🔬

> *"Ever watched a doctor diagnose a patient? They don't guess - they run tests, check vitals, and look at scans. Your GPU kernels deserve the same thorough examination."*

---

## 🎯 Learning Objectives

By the end of this session, you will:
1. **Profile** CUDA kernels using Nsight Compute command-line interface
2. **Interpret** key metrics: SM throughput, memory throughput, occupancy
3. **Identify** whether a kernel is compute-bound or memory-bound
4. **Diagnose** common performance bottlenecks from profiler output
5. **Generate** and analyze profiler reports for optimization guidance

---

## 1. Introduction to Nsight Compute

### What is Nsight Compute?

Nsight Compute is NVIDIA's **kernel-level profiler** that provides:
- Detailed metrics for each kernel launch
- Source-level analysis
- Roofline analysis
- Guided optimization recommendations

### When to Use It

| Tool | Use Case |
|------|----------|
| **Nsight Compute** | Optimizing individual kernels |
| **Nsight Systems** | Understanding application timeline, CPU-GPU interaction |

### Installation Check

```bash
# Check if ncu is installed
ncu --version

# Typical output:
# NVIDIA (R) Nsight Compute Command Line Profiler
# Copyright (c) 2018-2024 NVIDIA Corporation
# Version 2024.1.0.0
```

## 💡 Concept Card: The Kernel Microscope

```
┌─────────────────────────────────────────────────────────────────┐
│              NSIGHT COMPUTE = KERNEL MICROSCOPE                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   🔬 Just like a microscope reveals cellular details...         │
│                                                                 │
│   Your Code        →  What You See                              │
│   ──────────────────────────────────────────────────────────── │
│   kernel<<<...>>>  →  "It runs in 5ms"                          │
│                                                                 │
│   🔬 Nsight Compute reveals the INTERNAL machinery:             │
│                                                                 │
│   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐            │
│   │ SM Activity │  │Memory Flow  │  │ Occupancy   │            │
│   │ 45% compute │  │ 80% BW used │  │ 62.5% SMs   │            │
│   └─────────────┘  └─────────────┘  └─────────────┘            │
│                                                                 │
│   THE DIAGNOSIS ANALOGY:                                        │
│   ┌────────────────────────────────────────────────────────┐   │
│   │  Doctor               │  Nsight Compute                │   │
│   ├────────────────────────────────────────────────────────┤   │
│   │  Blood pressure       │  Memory throughput             │   │
│   │  Heart rate           │  SM throughput                 │   │
│   │  Temperature          │  Occupancy                     │   │
│   │  X-ray/MRI            │  Source correlation            │   │
│   │  Prescription         │  Optimization recommendations  │   │
│   └────────────────────────────────────────────────────────┘   │
│                                                                 │
│   💊 Don't guess - PROFILE! Let the data guide optimization.   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Week 7 Connection**: We measured kernel time. Now we see *inside* that time.

### 🔷 CUDA C++ Implementation (Primary)

## 2. CUDA C++ Test Kernels

Let's create kernels with different performance characteristics to profile. The code below demonstrates:
- **Memory-Bound**: Simple copy kernel
- **Compute-Bound**: Heavy math operations
- **Non-Coalesced**: Column-major access pattern
- **Bank Conflicts**: Naive reduction with conflicts
- **Low Occupancy**: High register usage

In [ ]:
%%writefile profiling_targets.cu
// profiling_targets.cu - Kernels to profile
#include <cuda_runtime.h>
#include <stdio.h>
#include <chrono>

// Utility macro
#define CHECK_CUDA(call) { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, \
               cudaGetErrorString(err)); \
        exit(1); \
    } \
}

//=============================================================================
// Kernel 1: Memory-Bound (Simple Copy)
//=============================================================================
__global__ void copyKernel(float* dst, const float* src, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        dst[idx] = src[idx];
    }
}

//=============================================================================
// Kernel 2: Compute-Bound (Heavy Math)
//=============================================================================
__global__ void computeKernel(float* output, const float* input, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = input[idx];
        
        // Heavy compute: transcendentals and iterations
        #pragma unroll 10
        for (int i = 0; i < 100; i++) {
            val = sinf(val) + cosf(val);
            val = sqrtf(fabsf(val) + 1.0f);
            val = expf(-val * 0.001f);
        }
        
        output[idx] = val;
    }
}

//=============================================================================
// Kernel 3: Non-Coalesced Access (Column Major)
//=============================================================================
__global__ void nonCoalescedKernel(float* dst, const float* src, 
                                    int rows, int cols) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < rows * cols) {
        int row = idx / cols;
        int col = idx % cols;
        
        // Column-major read (non-coalesced)
        int src_idx = col * rows + row;
        dst[idx] = src[src_idx];
    }
}

//=============================================================================
// Kernel 4: Bank Conflicts (Naive Reduction)
//=============================================================================
__global__ void bankConflictReduction(float* output, const float* input, int n) {
    __shared__ float sdata[256];
    
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? input[idx] : 0.0f;
    __syncthreads();
    
    // Naive reduction with bank conflicts
    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        if (tid % (2 * stride) == 0) {
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        output[blockIdx.x] = sdata[0];
    }
}

//=============================================================================
// Kernel 5: Low Occupancy (High Register Usage)
//=============================================================================
__global__ void lowOccupancyKernel(float* output, const float* input, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Force high register usage
    float r0 = input[idx % n], r1 = r0, r2 = r0, r3 = r0;
    float r4 = r0, r5 = r0, r6 = r0, r7 = r0;
    float r8 = r0, r9 = r0, r10 = r0, r11 = r0;
    float r12 = r0, r13 = r0, r14 = r0, r15 = r0;
    float r16 = r0, r17 = r0, r18 = r0, r19 = r0;
    float r20 = r0, r21 = r0, r22 = r0, r23 = r0;
    float r24 = r0, r25 = r0, r26 = r0, r27 = r0;
    float r28 = r0, r29 = r0, r30 = r0, r31 = r0;
    
    // Keep all registers live
    for (int i = 0; i < 10; i++) {
        r0 = r1 + r2; r1 = r2 + r3; r2 = r3 + r4; r3 = r4 + r5;
        r4 = r5 + r6; r5 = r6 + r7; r6 = r7 + r8; r7 = r8 + r9;
        r8 = r9 + r10; r9 = r10 + r11; r10 = r11 + r12; r11 = r12 + r13;
        r12 = r13 + r14; r13 = r14 + r15; r14 = r15 + r16; r15 = r16 + r17;
        r16 = r17 + r18; r17 = r18 + r19; r18 = r19 + r20; r19 = r20 + r21;
        r20 = r21 + r22; r21 = r22 + r23; r22 = r23 + r24; r23 = r24 + r25;
        r24 = r25 + r26; r25 = r26 + r27; r26 = r27 + r28; r27 = r28 + r29;
        r28 = r29 + r30; r29 = r30 + r31; r30 = r31 + r0; r31 = r0 + r1;
    }
    
    if (idx < n) {
        output[idx] = r0 + r1 + r2 + r3 + r4 + r5 + r6 + r7 +
                      r8 + r9 + r10 + r11 + r12 + r13 + r14 + r15 +
                      r16 + r17 + r18 + r19 + r20 + r21 + r22 + r23 +
                      r24 + r25 + r26 + r27 + r28 + r29 + r30 + r31;
    }
}

//=============================================================================
// Main - Run all kernels
//=============================================================================
int main() {
    const int N = 1 << 22;  // 4M elements
    const int ROWS = 4096;
    const int COLS = 1024;
    
    // Allocate memory
    float *d_input, *d_output, *d_src, *d_dst;
    CHECK_CUDA(cudaMalloc(&d_input, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_output, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_src, ROWS * COLS * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_dst, ROWS * COLS * sizeof(float)));
    
    // Initialize
    float* h_input = new float[N];
    for (int i = 0; i < N; i++) h_input[i] = (float)(i % 1000) / 1000.0f;
    CHECK_CUDA(cudaMemcpy(d_input, h_input, N * sizeof(float), 
                          cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_src, h_input, ROWS * COLS * sizeof(float),
                          cudaMemcpyHostToDevice));
    
    int blockSize = 256;
    int gridSize = (N + blockSize - 1) / blockSize;
    int gridSize2D = (ROWS * COLS + blockSize - 1) / blockSize;
    
    printf("Running profiling target kernels...\n\n");
    
    // Run each kernel
    printf("1. Copy Kernel (Memory-Bound)\n");
    copyKernel<<<gridSize, blockSize>>>(d_output, d_input, N);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    printf("2. Compute Kernel (Compute-Bound)\n");
    computeKernel<<<gridSize, blockSize>>>(d_output, d_input, N);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    printf("3. Non-Coalesced Kernel\n");
    nonCoalescedKernel<<<gridSize2D, blockSize>>>(d_dst, d_src, ROWS, COLS);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    printf("4. Bank Conflict Reduction\n");
    bankConflictReduction<<<gridSize, blockSize>>>(d_output, d_input, N);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    printf("5. Low Occupancy Kernel\n");
    lowOccupancyKernel<<<gridSize, blockSize>>>(d_output, d_input, N);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    printf("\nAll kernels complete. Use ncu to profile!\n");
    
    // Cleanup
    delete[] h_input;
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_src);
    cudaFree(d_dst);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 -lineinfo -o profiling_targets profiling_targets.cu
!./profiling_targets

## 3. Basic Nsight Compute Usage

### Command Line Profiling

```bash
# Basic profiling - profile all kernels
ncu ./profiling_targets

# Profile specific kernel by name
ncu --kernel-name copyKernel ./profiling_targets

# Profile specific kernel by launch number
ncu --launch-skip 0 --launch-count 1 ./profiling_targets

# Save report to file
ncu -o profile_report ./profiling_targets
# Creates profile_report.ncu-rep (open in Nsight Compute GUI)
```

### Profiling Sections

```bash
# Full analysis (all sections)
ncu --set full ./profiling_targets

# Roofline analysis only
ncu --set roofline ./profiling_targets

# Memory analysis
ncu --section MemoryWorkloadAnalysis ./profiling_targets

# Compute analysis
ncu --section ComputeWorkloadAnalysis ./profiling_targets

# Occupancy analysis
ncu --section Occupancy ./profiling_targets
```

## 4. Key Metrics Explained

### Throughput Metrics

| Metric | Description | Good Value |
|--------|-------------|------------|
| **SM Throughput** | % of SM compute cycles used | >60% compute-bound |
| **Memory Throughput** | % of peak memory bandwidth | >60% memory-bound |
| **L1 Hit Rate** | Cache hit rate for L1 | Higher = better |
| **L2 Hit Rate** | Cache hit rate for L2 | Higher = better |

### Occupancy Metrics

| Metric | Description |
|--------|-------------|
| **Theoretical Occupancy** | Max warps per SM / hardware limit |
| **Achieved Occupancy** | Actual average active warps |
| **Active Warps** | Number of resident warps |

### Memory Metrics

| Metric | Description |
|--------|-------------|
| **Global Load Efficiency** | Useful bytes / total bytes loaded |
| **Global Store Efficiency** | Useful bytes / total bytes stored |
| **Shared Bank Conflicts** | Conflicts per shared memory access |

### Interpreting Results

```
Example ncu output for copyKernel:

Section: GPU Speed Of Light Throughput
  DRAM Throughput             95.2%
  SM Throughput               12.3%

Interpretation:
- High DRAM (95.2%) + Low SM (12.3%) = MEMORY-BOUND
- This is expected for a simple copy kernel
- Optimization: focus on memory access patterns, not compute
```

## 5. Practical Profiling Workflow

### Step 1: Quick Overview

```bash
# Get quick summary of all kernels
ncu --target-processes all ./profiling_targets 2>&1 | head -100
```

### Step 2: Identify Bottleneck Type

```bash
# Focus on speed-of-light analysis
ncu --section SpeedOfLight --kernel-name computeKernel ./profiling_targets
```

Expected output pattern:
```
copyKernel:    Memory ~95%, Compute ~12%  → Memory-bound
computeKernel: Memory ~15%, Compute ~85%  → Compute-bound
```

### Step 3: Deep Dive Based on Bottleneck

**For memory-bound kernels:**
```bash
ncu --section MemoryWorkloadAnalysis \
    --section MemoryWorkloadAnalysis_Chart \
    --kernel-name copyKernel ./profiling_targets
```

**For compute-bound kernels:**
```bash
ncu --section ComputeWorkloadAnalysis \
    --section WarpStateStatistics \
    --kernel-name computeKernel ./profiling_targets
```

### Step 4: Check for Common Issues

```bash
# Check for non-coalesced access
ncu --metrics l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum,\
l1tex__t_requests_pipe_lsu_mem_global_op_ld.sum \
    --kernel-name nonCoalescedKernel ./profiling_targets

# Calculate coalescing efficiency:
# Efficiency = requests / sectors
# Perfect = 1.0, Non-coalesced < 1.0
```

## 6. Specific Metric Queries

### Custom Metric Collection

```bash
# Occupancy metrics
ncu --metrics sm__warps_active.avg.pct_of_peak_sustained_active \
    ./profiling_targets

# Memory throughput
ncu --metrics dram__bytes.sum.per_second \
    ./profiling_targets

# Compute throughput
ncu --metrics sm__sass_thread_inst_executed_op_fadd_pred_on.sum,\
sm__sass_thread_inst_executed_op_fmul_pred_on.sum,\
sm__sass_thread_inst_executed_op_ffma_pred_on.sum \
    ./profiling_targets

# Shared memory bank conflicts
ncu --metrics l1tex__data_bank_conflicts_pipe_lsu_mem_shared.sum \
    --kernel-name bankConflictReduction ./profiling_targets
```

### Common Useful Metrics

```bash
# One-liner for key metrics
ncu --metrics \
sm__throughput.avg.pct_of_peak_sustained_elapsed,\
dram__throughput.avg.pct_of_peak_sustained_elapsed,\
sm__warps_active.avg.pct_of_peak_sustained_active,\
l1tex__t_sector_hit_rate.pct \
    ./profiling_targets
```

### 🔷 CUDA C++ Implementation (Primary)

## 7. Comparing Kernel Versions

The following code compares naive vs optimized matrix transpose:
- **Naive transpose**: Non-coalesced writes to global memory
- **Optimized transpose**: Uses shared memory tile with padding to avoid bank conflicts

In [ ]:
%%writefile kernel_comparison.cu
// kernel_comparison.cu - Compare optimized vs naive
#include <cuda_runtime.h>
#include <stdio.h>

// Naive transpose (non-coalesced writes)
__global__ void transposeNaive(float* out, const float* in, 
                                int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x < width && y < height) {
        out[x * height + y] = in[y * width + x];  // Non-coalesced write
    }
}

// Optimized transpose (coalesced with shared memory)
#define TILE_DIM 32
#define BLOCK_ROWS 8

__global__ void transposeCoalesced(float* out, const float* in,
                                    int width, int height) {
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];  // +1 for bank conflicts
    
    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;
    
    // Coalesced read into shared memory
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < width && (y + j) < height) {
            tile[threadIdx.y + j][threadIdx.x] = in[(y + j) * width + x];
        }
    }
    
    __syncthreads();
    
    // Coalesced write from shared memory
    x = blockIdx.y * TILE_DIM + threadIdx.x;  // Transpose block position
    y = blockIdx.x * TILE_DIM + threadIdx.y;
    
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (x < height && (y + j) < width) {
            out[(y + j) * height + x] = tile[threadIdx.x][threadIdx.y + j];
        }
    }
}

int main() {
    const int WIDTH = 4096;
    const int HEIGHT = 4096;
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, WIDTH * HEIGHT * sizeof(float));
    cudaMalloc(&d_out, WIDTH * HEIGHT * sizeof(float));
    
    dim3 blockNaive(32, 32);
    dim3 gridNaive((WIDTH + 31) / 32, (HEIGHT + 31) / 32);
    
    dim3 blockCoalesced(TILE_DIM, BLOCK_ROWS);
    dim3 gridCoalesced((WIDTH + TILE_DIM - 1) / TILE_DIM,
                       (HEIGHT + TILE_DIM - 1) / TILE_DIM);
    
    printf("Running naive transpose...\n");
    transposeNaive<<<gridNaive, blockNaive>>>(d_out, d_in, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    printf("Running coalesced transpose...\n");
    transposeCoalesced<<<gridCoalesced, blockCoalesced>>>(d_out, d_in, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    printf("Done. Compare with ncu!\n");
    
    cudaFree(d_in);
    cudaFree(d_out);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 -lineinfo -o kernel_comparison kernel_comparison.cu
!./kernel_comparison

### 🔶 Python/Numba (Optional - Quick Testing)

## 8. Python/Numba Optional Backup

Since Nsight Compute is a command-line tool, we can use Python for simulating profiling concepts:

In [ ]:
# Install dependencies
!pip install numba numpy matplotlib -q

In [ ]:
import numpy as np
import time
from numba import cuda
import math

# Check GPU availability
if cuda.is_available():
    device = cuda.get_current_device()
    print(f"GPU: {device.name}")
    print(f"Compute Capability: {device.compute_capability}")
    print(f"Total Memory: {device.total_memory / 1e9:.2f} GB")
else:
    print("No CUDA GPU available")

In [ ]:
# Simulate profiling by timing different kernel types

@cuda.jit
def copy_kernel(dst, src):
    """Memory-bound kernel - simple copy"""
    idx = cuda.grid(1)
    if idx < src.size:
        dst[idx] = src[idx]

@cuda.jit
def compute_kernel(output, input_arr):
    """Compute-bound kernel - heavy math"""
    idx = cuda.grid(1)
    if idx < input_arr.size:
        val = input_arr[idx]
        for i in range(100):
            val = math.sin(val) + math.cos(val)
            val = math.sqrt(abs(val) + 1.0)
            val = math.exp(-val * 0.001)
        output[idx] = val

# Test data
N = 1 << 20  # 1M elements
h_input = np.random.rand(N).astype(np.float32)
h_output = np.zeros_like(h_input)

# Device arrays
d_input = cuda.to_device(h_input)
d_output = cuda.device_array_like(h_input)

# Launch config
threads_per_block = 256
blocks_per_grid = (N + threads_per_block - 1) // threads_per_block

# Warm up
copy_kernel[blocks_per_grid, threads_per_block](d_output, d_input)
cuda.synchronize()

# Time copy kernel (memory-bound)
start = time.perf_counter()
for _ in range(100):
    copy_kernel[blocks_per_grid, threads_per_block](d_output, d_input)
cuda.synchronize()
copy_time = (time.perf_counter() - start) / 100

# Time compute kernel (compute-bound)  
start = time.perf_counter()
for _ in range(10):
    compute_kernel[blocks_per_grid, threads_per_block](d_output, d_input)
cuda.synchronize()
compute_time = (time.perf_counter() - start) / 10

# Analysis
bytes_transferred = 2 * N * 4  # Read + Write, float32
copy_bandwidth = bytes_transferred / copy_time / 1e9

print(f"Copy Kernel:")
print(f"  Time: {copy_time*1000:.3f} ms")
print(f"  Effective Bandwidth: {copy_bandwidth:.2f} GB/s")
print(f"  → Memory-bound (simple memory operations)")

print(f"\nCompute Kernel:")
print(f"  Time: {compute_time*1000:.3f} ms")
print(f"  Time Ratio (Compute/Copy): {compute_time/copy_time:.1f}x")
print(f"  → Compute-bound (heavy math operations)")

## 9. Key Takeaways

### Nsight Compute Essentials

1. **Basic profiling**: `ncu ./my_program`
2. **Kernel selection**: `--kernel-name KernelName`
3. **Save reports**: `-o report_name`
4. **Key sections**:
   - `SpeedOfLight` - Quick bottleneck identification
   - `MemoryWorkloadAnalysis` - Memory access patterns
   - `ComputeWorkloadAnalysis` - Compute utilization
   - `Occupancy` - Thread parallelism

### Bottleneck Identification

| SM Throughput | Memory Throughput | Bottleneck |
|---------------|-------------------|------------|
| High | Low | Compute-bound |
| Low | High | Memory-bound |
| Low | Low | Latency-bound or low occupancy |

### Best Practices

1. **Compile with `-lineinfo`** for source correlation
2. **Start with overview**, then deep dive
3. **Compare before/after** optimization
4. **Focus on one bottleneck at a time**

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile nsight_compute_exercises.cu
/*
 * Nsight Compute Exercises
 * Week 8, Day 1: Kernel-Level Profiling Exercises
 * 
 * Profile these kernels with ncu to analyze performance metrics.
 * 
 * Compile: nvcc -arch=sm_75 -o nsight_compute_exercises nsight_compute_exercises.cu
 * Profile: ncu --set detailed ./nsight_compute_exercises
 */

#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA Error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
            exit(1); \
        } \
    } while(0)

#define N (1 << 22)  // 4M elements
#define BLOCK_SIZE 256

// =============================================================================
// Exercise 1: Coalesced vs Non-Coalesced Memory Access
// =============================================================================
// Profile these kernels and compare global load/store efficiency

__global__ void coalescedCopy(float* dst, const float* src, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        dst[idx] = src[idx];  // Coalesced access
    }
}

__global__ void stridedCopy(float* dst, const float* src, int n, int stride) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int strided_idx = (idx * stride) % n;
    if (idx < n) {
        dst[idx] = src[strided_idx];  // Strided access - poor coalescing
    }
}

// =============================================================================
// Exercise 2: Occupancy Analysis
// =============================================================================
// Compare occupancy between high and low register usage kernels

// High occupancy kernel - minimal registers
__global__ void highOccupancyKernel(float* out, const float* in, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        out[idx] = in[idx] * 2.0f;
    }
}

// Low occupancy kernel - many registers
__global__ void lowOccupancyKernel(float* out, const float* in, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Force register pressure with local variables
        float r0 = in[idx];
        float r1 = r0 * 1.1f, r2 = r0 * 1.2f, r3 = r0 * 1.3f, r4 = r0 * 1.4f;
        float r5 = r1 * 1.1f, r6 = r2 * 1.2f, r7 = r3 * 1.3f, r8 = r4 * 1.4f;
        float r9 = r5 * 1.1f, r10 = r6 * 1.2f, r11 = r7 * 1.3f, r12 = r8 * 1.4f;
        float r13 = r9 * 1.1f, r14 = r10 * 1.2f, r15 = r11 * 1.3f, r16 = r12 * 1.4f;
        out[idx] = r1 + r2 + r3 + r4 + r5 + r6 + r7 + r8 + 
                   r9 + r10 + r11 + r12 + r13 + r14 + r15 + r16;
    }
}

// =============================================================================
// Exercise 3: Shared Memory Bank Conflicts
// =============================================================================
// Analyze bank conflicts with ncu

#define TILE_SIZE 32

__global__ void noBankConflicts(float* out, const float* in, int n) {
    __shared__ float tile[TILE_SIZE][TILE_SIZE];
    
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;
    
    if (row < n && col < n) {
        tile[ty][tx] = in[row * n + col];  // Row-major - no conflicts
    }
    __syncthreads();
    
    if (row < n && col < n) {
        out[row * n + col] = tile[ty][tx];
    }
}

__global__ void withBankConflicts(float* out, const float* in, int n) {
    __shared__ float tile[TILE_SIZE][TILE_SIZE];
    
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;
    
    if (row < n && col < n) {
        tile[tx][ty] = in[row * n + col];  // Column access - bank conflicts!
    }
    __syncthreads();
    
    if (row < n && col < n) {
        out[row * n + col] = tile[tx][ty];
    }
}

// =============================================================================
// Exercise 4: Compute vs Memory Bound Analysis
// =============================================================================
// Profile and determine which is compute-bound vs memory-bound

__global__ void memoryBoundKernel(float* out, const float* in1, const float* in2, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        out[idx] = in1[idx] + in2[idx];  // Low arithmetic intensity
    }
}

__global__ void computeBoundKernel(float* out, const float* in, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = in[idx];
        // High compute with single load
        #pragma unroll
        for (int i = 0; i < 100; i++) {
            val = sinf(val) * cosf(val) + sqrtf(fabsf(val));
        }
        out[idx] = val;
    }
}

void runExercise1() {
    printf("\n=== Exercise 1: Memory Coalescing Analysis ===\n");
    printf("Profile command:\n");
    printf("  ncu --metrics l1tex__t_bytes_pipe_lsu_mem_global_op_ld.sum,");
    printf("l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum ");
    printf("--kernel-name-base demangled ./nsight_compute_exercises\n\n");
    
    float *d_src, *d_dst;
    CUDA_CHECK(cudaMalloc(&d_src, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_dst, N * sizeof(float)));
    
    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Coalesced copy
    coalescedCopy<<<blocks, BLOCK_SIZE>>>(d_dst, d_src, N);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ coalescedCopy completed\n");
    
    // Strided copy (stride=32 causes worst case)
    stridedCopy<<<blocks, BLOCK_SIZE>>>(d_dst, d_src, N, 32);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ stridedCopy completed\n");
    
    cudaFree(d_src);
    cudaFree(d_dst);
}

void runExercise2() {
    printf("\n=== Exercise 2: Occupancy Analysis ===\n");
    printf("Profile command:\n");
    printf("  ncu --metrics sm__warps_active.avg.pct_of_peak_sustained_active,");
    printf("launch__registers_per_thread ./nsight_compute_exercises\n\n");
    
    float *d_in, *d_out;
    CUDA_CHECK(cudaMalloc(&d_in, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_out, N * sizeof(float)));
    
    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    highOccupancyKernel<<<blocks, BLOCK_SIZE>>>(d_out, d_in, N);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ highOccupancyKernel completed\n");
    
    lowOccupancyKernel<<<blocks, BLOCK_SIZE>>>(d_out, d_in, N);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ lowOccupancyKernel completed\n");
    
    cudaFree(d_in);
    cudaFree(d_out);
}

void runExercise3() {
    printf("\n=== Exercise 3: Shared Memory Bank Conflicts ===\n");
    printf("Profile command:\n");
    printf("  ncu --metrics l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_ld.sum,");
    printf("l1tex__data_bank_conflicts_pipe_lsu_mem_shared_op_st.sum ");
    printf("./nsight_compute_exercises\n\n");
    
    int MATRIX_SIZE = 1024;
    float *d_in, *d_out;
    CUDA_CHECK(cudaMalloc(&d_in, MATRIX_SIZE * MATRIX_SIZE * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_out, MATRIX_SIZE * MATRIX_SIZE * sizeof(float)));
    
    dim3 blocks2D((MATRIX_SIZE + TILE_SIZE - 1) / TILE_SIZE, 
                  (MATRIX_SIZE + TILE_SIZE - 1) / TILE_SIZE);
    dim3 threads2D(TILE_SIZE, TILE_SIZE);
    
    noBankConflicts<<<blocks2D, threads2D>>>(d_out, d_in, MATRIX_SIZE);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ noBankConflicts completed\n");
    
    withBankConflicts<<<blocks2D, threads2D>>>(d_out, d_in, MATRIX_SIZE);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ withBankConflicts completed\n");
    
    cudaFree(d_in);
    cudaFree(d_out);
}

void runExercise4() {
    printf("\n=== Exercise 4: Compute vs Memory Bound ===\n");
    printf("Profile command:\n");
    printf("  ncu --metrics sm__throughput.avg.pct_of_peak_sustained_elapsed,");
    printf("gpu__dram_throughput.avg.pct_of_peak_sustained_elapsed ");
    printf("./nsight_compute_exercises\n\n");
    
    float *d_in1, *d_in2, *d_out;
    CUDA_CHECK(cudaMalloc(&d_in1, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_in2, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_out, N * sizeof(float)));
    
    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    memoryBoundKernel<<<blocks, BLOCK_SIZE>>>(d_out, d_in1, d_in2, N);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ memoryBoundKernel completed (expect high memory throughput %%)\n");
    
    computeBoundKernel<<<blocks, BLOCK_SIZE>>>(d_out, d_in1, N);
    CUDA_CHECK(cudaDeviceSynchronize());
    printf("✓ computeBoundKernel completed (expect high SM throughput %%)\n");
    
    cudaFree(d_in1);
    cudaFree(d_in2);
    cudaFree(d_out);
}

int main() {
    printf("Nsight Compute Profiling Exercises\n");
    printf("===================================\n");
    
    cudaDeviceProp prop;
    CUDA_CHECK(cudaGetDeviceProperties(&prop, 0));
    printf("Device: %s\n", prop.name);
    printf("Compute Capability: %d.%d\n", prop.major, prop.minor);
    
    runExercise1();
    runExercise2();
    runExercise3();
    runExercise4();
    
    printf("\n=== Profile the full application ===\n");
    printf("Full analysis: ncu --set full -o ncu_report ./nsight_compute_exercises\n");
    printf("Open in GUI:   ncu-ui ncu_report.ncu-rep\n");
    printf("\nAll exercises completed successfully!\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o nsight_compute_exercises nsight_compute_exercises.cu && ./nsight_compute_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises use Python with Numba for profiling concepts. While the CUDA C++ exercises above are the primary learning material, these provide an alternative approach using Python.

## 🎯 Summary & Key Takeaways

### What You Learned Today

| Concept | Key Insight |
|---------|-------------|
| **Kernel Microscope** | Nsight Compute reveals internal kernel behavior |
| **SM Throughput** | How busy are your compute units? |
| **Memory Throughput** | How saturated is memory bandwidth? |
| **Occupancy** | Are enough warps hiding latency? |
| **Bound Detection** | Compare SM vs Memory % to find bottleneck |

### 📋 Nsight Compute Profiling Checklist

```
┌─────────────────────────────────────────────────────────────────┐
│               KERNEL PROFILING CHECKLIST                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  □ Run baseline:  ncu ./myapp                                   │
│  □ Check SM throughput percentage (compute saturation)          │
│  □ Check Memory throughput percentage (bandwidth saturation)    │
│  □ Check Occupancy (warp scheduling efficiency)                 │
│  □ Identify bottleneck: Higher % = likely bottleneck            │
│                                                                 │
│  DIAGNOSIS QUICK REFERENCE:                                     │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │  SM High, Mem Low   → Compute-bound (more math!)        │   │
│  │  SM Low, Mem High   → Memory-bound (optimize access)    │   │
│  │  Both Low           → Latency-bound (increase occupancy)│   │
│  │  Both High          → Great! Near peak performance      │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  □ Use --section SpeedOfLight for quick summary                 │
│  □ Save report: ncu -o report.ncu-rep ./myapp                   │
│  □ Open in GUI for source correlation                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## 🔮 What's Next?

**Day 2: Roofline Analysis** - We'll visualize performance limits:
- Where is your kernel on the roofline?
- Are you hitting the compute ceiling or memory ceiling?
- The roofline model makes optimization strategy crystal clear!

*The microscope shows the details; the roofline shows the limits.*